In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\divya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\divya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# words = []
# classes = []
# documents = []
# ignore_words = ['?','!']
# data_file = open('intent.json').read()
# intents = json.loads(data_file)

# for intent in intents['intents']:
#     for pattern in intent['patterns']:

#         w = nltk.word_tokenize(pattern)
#         words.extend(w)
#         #adding documents
#         documents.append((w, intent['tag']))


#         #adding classes to our class list
#         if intent['tag'] not in classes:
#             classes.append(intent['tag'])

# words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
# words = sorted(list(set(words)))

# classes = sorted(list(set(classes)))

# print(len(documents), "documents")

# print(len(classes), "classes", classes)

# print(len(words), "unique lemmatized words", words)

# pickle.dump(words, open('words.pkl', 'wb'))
# pickle.dump(classes, open('classes.pkl', 'wb'))

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?','!']
data_file = open('intent.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for text in intent['text']:

        w = nltk.word_tokenize(text)
        words.extend(w)
        #adding documents
        documents.append((w, intent['intent']))


        #adding classes to our class list
        if intent['intent'] not in classes:
            classes.append(intent['intent'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words2.pkl', 'wb'))
pickle.dump(classes, open('classes2.pkl', 'wb'))

143 documents
22 classes ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye', 'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'WhoAmI']
117 unique lemmatized words ["'s", ',', 'a', 'about', 'adam', 'adios', 'am', 'any', 'anyone', 'are', 'aware', 'bay', 'be', 'bella', 'bored', 'by', 'bye', 'call', 'camera', 'can', 'cheering', 'clever', 'communicating', 'comprendo', 'conscious', 'could', 'do', 'doing', 'door', 'enough', 'for', 'friend', 'fuck', 'genious', 'get', 'girl', 'give', 'good', 'goodbye', 'gossip', 'got', 'great', 'have', 'hear', 'hello', 'help', 'helpful', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'identify', 'in', 'intelligent', 'is', 'it', 'joke', 'know', 'later', 'laugh', 'make', 'me', 'mean', 'meant', 'more', 'my', "n't", 'name', 'need', 'not', 'off', 'ok', 'op

In [7]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    #initializing bag of words
    bag = []

    #list of tokenized words for the pattern
    pattern_words = doc[0]
    #lemmatize each word - create base word
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    #create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)


    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_X = list(training[:, 0])
train_Y = list(training[:, 1])
print("Training data generated")

Training data generated


C:\Users\divya\AppData\Local\Temp\ipykernel_16884\1706977031.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape = (len(train_X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay = 1e-6, momentum=0.9, nesterov=True)
model.compile(loss = "categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_X), np.array(train_Y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbot_model.h5', hist)
print("Model created")

In [9]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words2.pkl', 'rb'))
classes = pickle.load(open('classes2.pkl', 'rb'))

In [39]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

#return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details = True):
    #tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    #bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                #assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in the bag: %s %w")
    return np.array(bag)

def predict_class(sentence, model):
    #filter out predictions below a threshold
    p = bow(sentence, words, show_details = False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    #sort by strength of probablity
    results.sort(key=lambda x: x[1], reverse = True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents['intents']
    for i in list_of_intents:
        if(['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
ints = predict_class("hello", model)
res = getResponse(ints, intents)

tag = ints[0]['intent']
list_of_intents = intents['intents']
for i in list_of_intents:
        if(['tag']== tag):
                result = random.choice(i['responses'])
        print(result)

In [40]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 21ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\divya\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\divya\AppData\Local\Temp\ipykernel_16884\3745697169.py", line 15, in send
    res = chatbot_response(msg)
  File "C:\Users\divya\AppData\Local\Temp\ipykernel_16884\3495194281.py", line 47, in chatbot_response
    res = getResponse(ints, intents)
  File "C:\Users\divya\AppData\Local\Temp\ipykernel_16884\3495194281.py", line 43, in getResponse
    return result
NameError: name 'result' is not defined
